# Generate mock data for a cluster

In this example we generate mock data with a variety of systematic effects including photometric redshifts, source galaxy distributions, and shape noise.  We then populate a galaxy cluster object. This notebooks is organised as follows:
- Imports and configuration setup
- Generate mock data with different source galaxy options
- Generate mock data with different field-of-view options
- Generate mock data with different galaxy cluster options (only available with the Numcosmo and/or CCL backends). Use the `os.environ['CLMM_MODELING_BACKEND']` line below to select your backend.

In [ ]:
import os

## Uncomment the following line if you want to use a specific modeling backend among 'ct' (cluster-toolkit), 'ccl' (CCL) or 'nc' (Numcosmo). Default is 'ccl'
# os.environ['CLMM_MODELING_BACKEND'] = 'nc'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


In [ ]:
import clmm

Make sure we know which version we're using

In [ ]:
clmm.__version__

## Import mock data module and setup the configuration 

In [ ]:
from clmm.support import mock_data as mock
from clmm import Cosmology

Mock data generation requires a defined cosmology

In [ ]:
mock_cosmo = Cosmology(H0=70.0, Omega_dm0=0.27 - 0.045, Omega_b0=0.045, Omega_k0=0.0)

Mock data generation requires some cluster information. The default is to work with the NFW profile, using the "200,mean" mass definition. The Numcosmo and CCL backends allow for more flexibility (see last section of this notebook)

In [ ]:
cosmo = mock_cosmo
cluster_id = "Awesome_cluster"
cluster_m = 1.0e15  # M200,m
cluster_z = 0.3
src_z = 0.8
concentration = 4
ngals = 1000  # number of source galaxies

# Cluster centre coordinates
cluster_ra = 50.0
cluster_dec = 87.0

It is also possible to choose the coordinate system for the generated ellipticities. Possible options are either "celestial" or "euclidean". The default choice is "euclidean". See https://doi.org/10.48550/arXiv.1407.7676 section 5.1 for more details.

In [ ]:
coordinate_system = "euclidean"

## Generate the mock catalog with different source galaxy options

- Clean data: no noise, all galaxies at the same redshift

In [ ]:
zsrc_min = cluster_z + 0.1

In [ ]:
ideal_data = mock.generate_galaxy_catalog(
    cluster_m,
    cluster_z,
    concentration,
    cosmo,
    src_z,
    ngals=ngals,
    cluster_ra=cluster_ra,
    cluster_dec=cluster_dec,
    coordinate_system=coordinate_system,
)

In [ ]:
# let's put all these quantities in a single dictionary to facilitate clarity
cluster_kwargs = {
    "cluster_m": cluster_m,
    "cluster_z": cluster_z,
    "cluster_ra": cluster_ra,
    "cluster_dec": cluster_dec,
    "cluster_c": concentration,
    "cosmo": cosmo,
    "coordinate_system": coordinate_system,
}

In [ ]:
ideal_data = mock.generate_galaxy_catalog(**cluster_kwargs, zsrc=src_z, ngals=ngals)

- Noisy data: shape noise, all galaxies at the same redshift

In [ ]:
noisy_data_src_z = mock.generate_galaxy_catalog(
    **cluster_kwargs, zsrc=src_z, shapenoise=0.05, ngals=ngals
)

* Noisy data: shape noise plus measurement error, all galaxies at the same redshift

In [ ]:
noisy_data_src_z_e_err = mock.generate_galaxy_catalog(
    **cluster_kwargs, zsrc=src_z, shapenoise=0.05, mean_e_err=0.05, ngals=ngals
)

<div class="alert alert-warning">

**WARNING:** Experimental feature. Uncertainties are created by simply drawing random numbers near the value specified by `mean_e_err`. Use at your own risk. This will be improved in future releases.
    
</div>

- Noisy data: photo-z errors (and pdfs!), all galaxies at the same redshift. At present, the pdfs are generated by drawing a random value from a Normal distribution with mean `ztrue` and width given by `dz = (1+z)*photoz_sigma_unscaled`, and the pdf is this Normal distribution centered around `z` instead of `ztrue`.

In [ ]:
np.random.seed(41363)

noisy_data_photoz = mock.generate_galaxy_catalog(
    **cluster_kwargs,
    zsrc=src_z,
    shapenoise=0.05,
    photoz_sigma_unscaled=0.05,
    pz_bins=101,
    ngals=ngals
)

In [ ]:
noisy_data_photoz[:5]

* Changing ellipticity coordinate system: notice that $e_2$ changes sign!

In [ ]:
cluster_kwargs_celestial = cluster_kwargs.copy()
cluster_kwargs_celestial["coordinate_system"] = "celestial"

np.random.seed(41363)

noisy_data_photoz_celestial = mock.generate_galaxy_catalog(
    **cluster_kwargs_celestial,
    zsrc=src_z,
    shapenoise=0.05,
    photoz_sigma_unscaled=0.05,
    pz_bins=101,
    ngals=ngals
)

In [ ]:
noisy_data_photoz_celestial[:5]

In the default PDF storing (`pzpdf_type='shared_bins'`), the values of the PDF are added to the `pzpdf` column and the binning scheme is stored in the `pzpdf_info` attribute:

In [ ]:
noisy_data_photoz.pzpdf_info

In [ ]:
noisy_data_photoz[:5]

It is also possible to generate individual binning of the PDF with `pzpdf_type='individual_bins'`. In this case, both the bin values and the PDF are added to the `pzpdf` column:

In [ ]:
mock.generate_galaxy_catalog(
    **cluster_kwargs,
    zsrc=src_z,
    shapenoise=0.05,
    photoz_sigma_unscaled=0.05,
    pz_bins=101,
    ngals=ngals,
    pzpdf_type="individual_bins"
)[:5]

Note that if `pzpdf_type=None`, the pdf is not stored:

In [ ]:
mock.generate_galaxy_catalog(
    **cluster_kwargs,
    zsrc=src_z,
    shapenoise=0.05,
    photoz_sigma_unscaled=0.05,
    pz_bins=101,
    ngals=ngals,
    pzpdf_type=None
)[:5]

- Clean data: source galaxy redshifts drawn from a redshift distribution instead of fixed `src_z` value. Options are `chang13` for Chang et al. 2013 or `desc_srd` for the distribution given in the DESC Science Requirement Document. No shape noise or photoz errors.

In [ ]:
ideal_with_src_dist = mock.generate_galaxy_catalog(
    **cluster_kwargs, zsrc="chang13", zsrc_min=zsrc_min, zsrc_max=7.0, ngals=ngals
)

- Noisy data: galaxies following redshift distribution, redshift error, shape noise

In [ ]:
pzbins = np.linspace(0, 10, 1001)

In [ ]:
allsystematics = mock.generate_galaxy_catalog(
    **cluster_kwargs,
    zsrc="chang13",
    zsrc_min=zsrc_min,
    photoz_sigma_unscaled=0.05,
    ngals=ngals,
    pz_bins=pzbins,
)

In [ ]:
allsystematics2 = mock.generate_galaxy_catalog(
    **cluster_kwargs,
    zsrc="desc_srd",
    zsrc_min=zsrc_min,
    zsrc_max=7.0,
    photoz_sigma_unscaled=0.05,
    ngals=ngals,
    pz_bins=pzbins,
    shapenoise=0.05,
)

Sanity check: checking that no galaxies were originally drawn below zsrc_min, before photoz errors are applied (when relevant)

In [ ]:
print(
    f"""Number of galaxies below zsrc_min:
    ideal_data          : {np.sum(ideal_data['ztrue']<zsrc_min):5,}
    noisy_data_src_z    : {np.sum(noisy_data_src_z['ztrue']<zsrc_min):5,}
    noisy_data_photoz   : {np.sum(noisy_data_photoz['ztrue']<zsrc_min):5,}
    ideal_with_src_dist : {np.sum(ideal_with_src_dist['ztrue']<zsrc_min):5,}
    allsystematics      : {np.sum(allsystematics['ztrue']<zsrc_min):5,}
"""
)

### Inspect the catalog data

- Ideal catalog first entries: no noise on the shape measurement, all galaxies at z=0.8, no redshift errors (z = ztrue)

In [ ]:
for n in ideal_data.colnames:
    if n != "id":
        ideal_data[n].format = "%6.3e"
ideal_data[0:3].show_in_notebook()

- With photo-z errors

In [ ]:
for n in noisy_data_photoz.colnames:
    if n != "id":
        noisy_data_photoz[n].format = "%6.3e"
noisy_data_photoz[0:3].show_in_notebook()

- Histogram of the redshift distribution of background galaxies, for the true (originally drawn) redshift and the redshift once photoz errors have been added, and the stacked pdf. By construction no true redshift occurs below zsrc_min, but some 'observed' redshifts (i.e. including photoz errors) might be.

In [ ]:
plt.hist(
    allsystematics["z"],
    bins=50,
    alpha=0.3,
    density=True,
    label="measured z (i.e. including photoz error)",
)
plt.hist(allsystematics["ztrue"], bins=50, alpha=0.3, density=True, label="true z")
stacked_pdf = np.mean(allsystematics["pzpdf"], axis=0)
plt.plot(allsystematics.pzpdf_info["zbins"], stacked_pdf, "C3", label="stacked pdf")
plt.axvline(zsrc_min, color="k", label="requested zmin")
plt.xlabel("Source Redshift")
plt.ylabel("n(z)")
plt.legend()
plt.xlim(0, 5)
pass

In [ ]:
plt.hist(allsystematics["ztrue"], bins=50, alpha=0.3, label="true z")
plt.hist(allsystematics2["ztrue"], bins=50, alpha=0.3, label="true z")

## Populate a galaxy cluster object

In [ ]:
gc_object = clmm.GalaxyCluster(cluster_id, cluster_ra, cluster_dec, cluster_z, allsystematics)

From a `GalaxyCluster` object that has photoz information, `draw_gal_z_from_pdz` allows to generate `nobj` random redshifts of each galaxy in `galcat`, from its photoz pdf, and store the result in a new `zcol_out` column.

In [ ]:
z_random = gc_object.draw_gal_z_from_pdz(zcol_out="z_random", overwrite=False, nobj=1)

The plot below shows the "observed photoz pdf" (blue), centered on the "observed z" (red), the true redshift from which the shear where computed (green) and  a random redshift (orange) computed from the pdf

In [ ]:
# p(z) for one of the galaxies in the catalog,
galid = 0
plt.fill(gc_object.galcat.pzpdf_info["zbins"], allsystematics["pzpdf"][galid], alpha=0.3)
plt.plot(gc_object.galcat.pzpdf_info["zbins"], gc_object.galcat["pzpdf"][galid], label="Photoz pdf")

plt.axvline(gc_object.galcat["z"][galid], label="Observed z", color="red")
plt.axvline(gc_object.galcat["ztrue"][galid], label="True z", color="g")
plt.axvline(gc_object.galcat["z_random"][galid], label="Random z from pdf", color="orange")

plt.xlabel("Redshift")
plt.ylabel("Photo-z Probability Distribution")
plt.legend(loc=1)
plt.xlim(gc_object.galcat["z"][galid] - 0.5, gc_object.galcat["z"][galid] + 0.5)

Plot source galaxy ellipticities

In [ ]:
plt.scatter(gc_object.galcat["e1"], gc_object.galcat["e2"])

plt.xlim(-0.2, 0.2)
plt.ylim(-0.2, 0.2)
plt.xlabel("Ellipticity 1", fontsize="x-large")
plt.ylabel("Ellipticity 2", fontsize="x-large")

## Generate the mock data catalog with different field-of-view options

In the examples above, `ngals=1000` galaxies were simulated in a field corresponding to a 8 Mpc x 8 Mpc (proper distance) square box at the cluster redshift (this is the default). The user may however vary the field size and/or provide a galaxy density (instead of a number of galaxies). This is examplified below, using the `allsystematics` example.

- `ngals = 1000` in a 4 x 4 Mpc box. Asking for the same number of galaxies in a smaller field of view yields high galaxy density

In [ ]:
allsystematics2 = mock.generate_galaxy_catalog(
    **cluster_kwargs,
    zsrc="chang13",
    zsrc_min=zsrc_min,
    zsrc_max=7.0,
    shapenoise=0.05,
    photoz_sigma_unscaled=0.05,
    field_size=4,
    ngals=ngals
)

In [ ]:
plt.scatter(allsystematics["ra"], allsystematics["dec"], marker=".", label="default 8 x 8 Mpc FoV")
plt.scatter(allsystematics2["ra"], allsystematics2["dec"], marker=".", label="user-defined FoV")
plt.legend()

- Alternatively, the user may provide a galaxy density (here ~1 gal/arcmin2 to roughly match 1000 galaxies, given the configuration) and the number of galaxies to draw will automatically be adjusted to the box size.

In [ ]:
allsystematics3 = mock.generate_galaxy_catalog(
    **cluster_kwargs,
    zsrc="chang13",
    zsrc_min=zsrc_min,
    zsrc_max=7.0,
    shapenoise=0.05,
    photoz_sigma_unscaled=0.05,
    ngal_density=1.3,
)
print(f"Number of drawn galaxies = {len(allsystematics3)}")

In [ ]:
allsystematics4 = mock.generate_galaxy_catalog(
    **cluster_kwargs,
    zsrc="desc_srd",
    zsrc_min=zsrc_min,
    zsrc_max=7.0,
    shapenoise=0.05,
    photoz_sigma_unscaled=0.05,
    ngal_density=1.3,
)
print(f"Number of drawn galaxies = {len(allsystematics4)}")

In [ ]:
plt.scatter(allsystematics["ra"], allsystematics["dec"], marker=".", label="ngals = 1000")
plt.scatter(
    allsystematics3["ra"],
    allsystematics3["dec"],
    marker=".",
    label="ngal_density = 1 gal / arcmin2",
)
plt.legend()

## Generate mock data with different galaxy cluster options
WARNING: Available options depend on the modeling backend:
- Cluster-toolkit allows for other values of the overdensity parameter, but is retricted to working with the mean mass definition
- Both CCL and Numcosmo allow for different values of the overdensity parameter, but work with both the mean and critical mass definition
- Numcosmo further allows for the Einasto or Burkert density profiles to be used instead of the NFW profile



### Changing the overdensity parameter (all backend) - `delta_so` keyword (default = 200)

In [ ]:
allsystematics_500mean = mock.generate_galaxy_catalog(
    **cluster_kwargs,
    zsrc="chang13",
    delta_so=500,
    zsrc_min=zsrc_min,
    zsrc_max=7.0,
    shapenoise=0.05,
    photoz_sigma_unscaled=0.05,
    ngals=ngals
)

### Using the critical mass definition (Numcosmo and CCL only) - `massdef` keyword (default = 'mean')
WARNING: error will be raised if using the cluster-toolkit backend

In [ ]:
allsystematics_200critical = mock.generate_galaxy_catalog(
    **cluster_kwargs,
    zsrc="chang13",
    massdef="critical",
    zsrc_min=zsrc_min,
    zsrc_max=7.0,
    shapenoise=0.05,
    photoz_sigma_unscaled=0.05,
    ngals=ngals
)

### Changing the halo density profile (Numcosmo and CCL only) - `halo_profile_model` keyword (default = 'nfw')
WARNING: error will be raised if using the cluster-toolkit or CCL backends

In [ ]:
allsystematics_200m_einasto = mock.generate_galaxy_catalog(
    **cluster_kwargs,
    zsrc="chang13",
    halo_profile_model="einasto",
    zsrc_min=zsrc_min,
    zsrc_max=7.0,
    shapenoise=0.05,
    photoz_sigma_unscaled=0.05,
    ngals=ngals
)

In [ ]:
ideal_data

In [ ]:
from astropy import wcs
dx = -1.0 / 60.0
dy =  1.0 / 60.0

w = wcs.WCS(naxis = 2)

w.wcs.crval = [cluster_ra, cluster_dec]
w.wcs.cd    = np.array([[dx, 0.0], [0.0, dy]])
w.wcs.ctype = ['RA---TAN', 'DEC--TAN']
w.wcs.crpix = [0, 0]

coord = np.stack([ideal_data['ra'], ideal_data['dec']], axis = 1)
    
xy = w.wcs_world2pix(coord, 1)

x = xy[:,0]
y = xy[:,1]

In [ ]:
ltmin  =  0.0
ltmax  =  1.28
dlt    =  0.16

nbin = int((ltmax - ltmin + 1.0e-30) / dlt)

lt1 = np.linspace(ltmin, ltmax - dlt, num = nbin)
lt2 = np.linspace(ltmin + dlt, ltmax, num = nbin)

tbin = 10 ** (0.5 * (lt1 + lt2))

t  = np.sqrt(x * x + y * y)
lt = np.log10(t)

In [ ]:
cos2 = (x * x - y * y) / (t * t + 1.0e-30)
sin2 = 2.0 * x * y  / (t * t + 1.0e-30)

gt = (-1.0) * cos2 * ideal_data['e1'] - sin2 * ideal_data['e2']
gx = sin2 * ideal_data['e1'] - cos2 * ideal_data['e2']

In [ ]:
gt_ave = np.zeros(nbin)
gx_ave = np.zeros(nbin)
gt_sig = np.zeros(nbin)
gx_sig = np.zeros(nbin)
weight = np.ones(np.shape(gt)[0])
for j in range(nbin):
    mask = np.logical_and((lt > lt1[j]), (lt < lt2[j]))
    ct   = t[mask]
    cgt  = gt[mask]
    cgx  = gx[mask]
    cwei = weight[mask]

    wsum1 = np.sum(cwei)
    wsum2 = np.sum(cwei * cwei)

    gt_ave[j] = np.sum(cgt * cwei) / wsum1
    gx_ave[j] = np.sum(cgx * cwei) / wsum1

    gt_sig[j] = np.sqrt((np.sum(cgt * cgt * cwei) / wsum1) * (wsum2 / (wsum1 * wsum1)))
    gx_sig[j] = np.sqrt((np.sum(cgx * cgx * cwei) / wsum1) * (wsum2 / (wsum1 * wsum1)))

In [ ]:
import matplotlib as mpl

In [ ]:
xmin = 1.0
xmax = 20.0
ymin = -0.08
ymax = 0.15

plt.figure(figsize = (8, 6))


plt.plot(tbin * 1.005, gt_ave, '.', markersize = 10, color = 'r')
plt.errorbar(tbin * 1.005, gt_ave, yerr = gt_sig, fmt = 'none', ecolor = 'r', label = '_nolegend_')

plt.plot(tbin * 0.995, gx_ave, 'x', markersize = 10, color = 'b')
plt.errorbar(tbin * 0.995, gx_ave, yerr = gt_sig, fmt = 'none', ecolor = 'b', label = '_nolegend_')

plt.legend(['$\gamma_+$', '$\gamma_\\times$'], markerscale = 1, frameon = False, labelspacing = 0.3, numpoints = 1, loc = 'upper right')

plt.hlines(0, xmin, xmax, linestyles = 'dotted', color = 'black')
ax = plt.gca()
ax.set_xscale('log')

ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.1))
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.02))

plt.xlim([xmin,xmax])
plt.ylim([ymin,ymax])

plt.xlabel('$\\theta$ [arcmin]')
plt.ylabel('$\gamma_+$ or $\gamma_\\times$')
#plt.title('ideal data')

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def fit_sis(x, t_ein):
    return 0.5 * (t_ein / x)

# fit parameter t_ein (Einstein radius) with observed shear profile
tein, cov = curve_fit(fit_sis, tbin, gt_ave, sigma = gt_sig)

print('Einstein radius: %f arcmin' % tein[0])

In [ ]:
# conpute best-fit profile
tt1 = 10 ** np.linspace(-1.0, 2.0, num = 300)
yy1 = fit_sis(tt1, tein[0])

xmin = 1.0
xmax = 20.0
ymin = 0.005
ymax = 0.3

plt.figure(figsize = (8, 6))

plt.hlines(0, xmin, xmax, linestyles = 'dotted')

plt.plot(tt1, yy1, '-', color = 'b', lw = 2.)

plt.plot(tbin, gt_ave, '.', markersize = 10, color = 'r')
plt.errorbar(tbin, gt_ave, yerr = gt_sig, fmt = 'none', ecolor = 'r', label = '_nolegend_')

plt.legend(['SIS fit'], markerscale = 1, frameon = False, labelspacing = 0.3, numpoints = 1, loc = 'upper right')

ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')

plt.xlim([xmin,xmax])
plt.ylim([ymin,ymax])

plt.xlabel('$\\theta$ [arcmin]')
plt.ylabel('$\gamma_+$')
plt.title('ideal data')

In [ ]:
from astropy import constants as const
from astropy import units as u
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0 = 70, Om0 = 0.27 - 0.045)

In [ ]:
tein_rad = tein[0] * u.arcmin
dos = cosmo.angular_diameter_distance(src_z)
dol = cosmo.angular_diameter_distance(cluster_z)
dls = cosmo.angular_diameter_distance_z1z2(cluster_z, src_z)
sig = const.c * np.sqrt((tein_rad * dos).to(u.Mpc, u.dimensionless_angles()) / (4.0 * np.pi * dls))

print('velocity dispersion: ', sig.to(u.km/u.s))

In [ ]:
from colossus.halo import concentration
from colossus.halo import profile_nfw
from colossus.cosmology import cosmology
params = {'flat': True, 'H0': 70.0, 'Om0': 0.27 - 0.045, 'Ob0': 0.045, 'sigma8': 0.81, 'ns': 0.95}
cosmology.addCosmology('myCosmo', **params)
cosmo_c = cosmology.setCosmology('myCosmo')

In [ ]:
def calc_R200m(halo_mass,redshift=0.25):
    concen = concentration.concentration(halo_mass,'200m',redshift,model='diemer15')
   
    profile = profile_nfw.NFWProfile(M=halo_mass,mdef='200m',z=redshift,c=concen)
    R200m = profile.RDelta(redshift,'200m')
    R200m /= 1000. #ts: convert to Mpc/h                                                                                            
    return concen,R200m

In [ ]:
def convert_M200m_to_Mvir(halo_mass,redshift=0.25):
    concen = concentration.concentration(halo_mass,'200m',redshift,model='diemer15')
    profile = profile_nfw.NFWProfile(M=halo_mass,mdef='200m',z=redshift,c=concen)
    Rvir = profile.RDelta(redshift,'vir')
    Mvir = profile.enclosedMass(Rvir)
    return concen,Rvir/1000.,Mvir

In [ ]:
concen,R200m = calc_R200m(10**15.,redshift=cluster_z)
cvir,Rvir,Mvir = convert_M200m_to_Mvir(10**15.,redshift=cluster_z)

In [ ]:
G = 4.299*10**(-9) #unit:[Mpc(km/s)^2 Msun^-1]  
Mhalo = 10**15.
hubble = cosmo.H(cluster_z)
a_z = 1./(1.+cluster_z)
sigma = G*(Mhalo)/R200m/a_z**2. #TS:[(Mpc/h)^2]

In [ ]:
np.sqrt(sigma)